In [1]:
from main import Simplify
import os
import json
import re
import sys
import time

import cv2
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import torchvision as tv
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

from aegan import AEGAN

BATCH_SIZE = 32
LATENT_DIM = 32

device = "cpu"  # torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = tv.transforms.Compose([
    tv.transforms.RandomAffine(0, translate=(5 / 96, 5 / 96), fill=(255, 255, 255)),
    # tv.transforms.ColorJitter(hue=0.5),
    tv.transforms.RandomHorizontalFlip(p=0.5),
    # Simplify(),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5,), (0.5, 0.5, 0.5,))
])
dataset = ImageFolder(
    root="data_cleared",
    transform=transform
)
dataloader = DataLoader(dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        num_workers=1,
                        drop_last=True
                        )

noise_fn = lambda x: torch.randn((x, LATENT_DIM), device=device)

gan = AEGAN(
    LATENT_DIM,
    noise_fn,
    dataloader,
    device=device,
    batch_size=BATCH_SIZE,
    channels_cnt=3,
    colors_cnt=16,
    is_old=True
)

In [2]:
gan.load(os.path.join("data_cleared_linear_upsampling_colorspace_stolen_discriminator_results", "checkpoints"), 499)

In [28]:
import os

from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('tensorboard')

In [29]:
from pathlib import Path
from PIL import Image

gan.encoder.eval()

embeddings = []
sprites = []
metadata = []

for filename in os.listdir("data_cleared/sprites_rgb"):
    img = Image.open(Path() / "data_cleared/sprites_rgb" / filename)
    # as_array = np.array(img)
    as_tensor = tv.transforms.ToTensor()(img)[None, :, :, :]

    with torch.no_grad():
        embedding = gan.encoder(as_tensor)

    img.thumbnail([28, 28], Image.ANTIALIAS)
    embeddings.append(embedding[0])
    sprites.append(tv.transforms.ToTensor()(img))
    metadata.append(filename)
    # break

writer.add_embedding(torch.stack(embeddings), label_img=torch.stack(sprites), metadata=metadata)
writer.close()

In [25]:
writer.add_embedding(torch.stack(embeddings), label_img=torch.stack(sprites), metadata=torch.Tensor(range(len(embeddings))))
writer.close()

In [15]:
img.thumbnail([96, 96], Image.ANTIALIAS)
np.array(img).shape

AttributeError: 'NoneType' object has no attribute 'thumbnail'